In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset

In [5]:
class DataProcessor:
    def __init__(self, input_path, file_names):
        self.input_path = input_path
        self.file_names = file_names
        
    def read_files(self):
        self.data = {}
        print("Reading files...")
        for file in self.file_names:
            with open(self.input_path + file + '.txt', 'r') as f:
                self.data[file] = pd.read_csv(f, header=None, sep='\t')
        return self.data
    
    def print_shape(self):
        print("Files read:")
        for file in self.data:
            print(f"{file}: {self.data[file].shape}")
            
    def create_target_df(self):
        target_columns = ['Cooler_Condition', 'Valve_Condition', 
                        'Internal_Pump_Leakage', 'Hydraulic_Accumulator', 
                        'Stable_Flag']
        self.data['target'].columns = target_columns
        self.valve_condition = self.data['target']['Valve_Condition']
        #del self.data['target']
        return self.valve_condition

def process_data():
    input_path = "input_data/"
    file_names = [
        "ce", "cp", "eps1", "se", "vs1", 
        "fs1", "fs2", 
        "ps1", "ps2", "ps3", "ps4", "ps5", "ps6",
        "ts1", "ts2", "ts3", "ts4", "target"
    ]
    
    processor = DataProcessor(input_path, file_names)
    data = processor.read_files()
    processor.print_shape()
    df_target = processor.create_target_df()
    df_target = processor.valve_condition
    return data, df_target

data, df_target = process_data()

Reading files...
Files read:
ce: (2205, 60)
cp: (2205, 60)
eps1: (2205, 6000)
se: (2205, 60)
vs1: (2205, 60)
fs1: (2205, 600)
fs2: (2205, 600)
ps1: (2205, 6000)
ps2: (2205, 6000)
ps3: (2205, 6000)
ps4: (2205, 6000)
ps5: (2205, 6000)
ps6: (2205, 6000)
ts1: (2205, 60)
ts2: (2205, 60)
ts3: (2205, 60)
ts4: (2205, 60)
target: (2205, 5)


In [8]:
input_df = pd.concat([data['ce'], data['cp'], data['eps1'], data['se'], data['vs1'], 
                      data['fs1'], data['fs2'], data['ps1'], data['ps2'], data['ps3'], data['ps4'], 
                      data['ps5'], data['ps6'], data['ts1'], data['ts2'], data['ts3'], data['ts4']], axis=1)

In [18]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, accuracy_score
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import LabelEncoder




# Standardisiere die Eingabedaten
scaler = StandardScaler()
input_data_scaled = scaler.fit_transform(input_df)

label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(df_target)

# Splitte die Daten in Trainings- und Testdaten
X_train, X_test, y_train, y_test = train_test_split(input_data_scaled, y_encoded, test_size=0.2, random_state=42)

# Konvertiere in Tensoren für PyTorch
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

# Erstelle DataLoader für das Training
train_data = TensorDataset(X_train_tensor, y_train_tensor)
test_data = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_data, batch_size=32, shuffle=True)
test_loader = DataLoader(test_data, batch_size=32, shuffle=False)

# TimeMixer Modell Implementierung
class TimeMixer(nn.Module):
    def __init__(self, input_size, output_size, hidden_size=128, num_layers=3, dropout=0.2):
        super(TimeMixer, self).__init__()
        
        # Zeitmischungsschichten
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.fc2 = nn.Linear(hidden_size, hidden_size)
        self.fc3 = nn.Linear(hidden_size, output_size)
        self.dropout = nn.Dropout(dropout)
        
        # Aktivierungsfunktion
        self.relu = nn.ReLU()
    
    def forward(self, x):
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        return x

# Modell, Verlustfunktion und Optimierer
model = TimeMixer(input_size=X_train.shape[1], output_size=4)  # Output size = Anzahl Klassen (z. B. 2 für binär)
criterion = nn.CrossEntropyLoss()  # Verlustfunktion für Klassifikation
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Trainingsschleife
num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        
        # Forward Pass
        outputs = model(inputs)
        
        # Verlust berechnen
        loss = criterion(outputs, targets)
        
        # Backward Pass und Optimierung
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    print(f'Epoche {epoch+1}/{num_epochs}, Verlust: {running_loss / len(train_loader)}')

# Evaluierung mit Classification Report
model.eval()
all_preds = []
all_targets = []

with torch.no_grad():
    for inputs, targets in test_loader:
        outputs = model(inputs)
        
        # Vorhersagen in Klassen umwandeln
        preds = torch.argmax(outputs, axis=1)  # Maximale Wahrscheinlichkeit → Klasse
        all_preds.extend(preds.cpu().numpy())
        all_targets.extend(targets.cpu().numpy())

# Classification Report
print("Classification Report:")
print(classification_report(all_targets, all_preds))

# Optional: Genauigkeit berechnen
accuracy = accuracy_score(all_targets, all_preds)
print(f"Genauigkeit: {accuracy * 100:.2f}%")


Epoche 1/50, Verlust: 4.556509667209217
Epoche 2/50, Verlust: 2.498163526611669
Epoche 3/50, Verlust: 1.171979677996465
Epoche 4/50, Verlust: 0.6447216091411454
Epoche 5/50, Verlust: 0.5712702706888584
Epoche 6/50, Verlust: 0.4669345053178923
Epoche 7/50, Verlust: 0.5181969981640577
Epoche 8/50, Verlust: 0.33220890475370524
Epoche 9/50, Verlust: 0.35471872220348033
Epoche 10/50, Verlust: 0.23822542925232223
Epoche 11/50, Verlust: 0.24063675311793173
Epoche 12/50, Verlust: 0.2475860056916385
Epoche 13/50, Verlust: 0.2221258759299027
Epoche 14/50, Verlust: 0.18540272720149784
Epoche 15/50, Verlust: 0.13954871158681012
Epoche 16/50, Verlust: 0.14804131289877528
Epoche 17/50, Verlust: 0.28147868536939313
Epoche 18/50, Verlust: 0.21276527429154157
Epoche 19/50, Verlust: 0.16137418765785178
Epoche 20/50, Verlust: 0.137578582066843
Epoche 21/50, Verlust: 0.14777985878286667
Epoche 22/50, Verlust: 0.19733251671173743
Epoche 23/50, Verlust: 0.3783714731024312
Epoche 24/50, Verlust: 0.1680915912